<a href="https://colab.research.google.com/github/Geraldnvs/DESAFIO_ALURA/blob/main/Desafio_Alura_%26_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>